# FF7R Translation & Localization Script

## Setup and Initialization

In [ ]:
# Import Libraries
import os
import pandas as pd
from openai import AzureOpenAI

# Initialize Azure OpenAI client with key-based authentication
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")  
deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
   
client = AzureOpenAI(
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version="2024-05-01-preview",  
)

In [ ]:
# Test the client
prompt = "Describe Tifa Lockhart from Final Fantasy VII."

completion = client.chat.completions.create(
    model=deployment,
    messages=[
        {"role": "system", "content": "You are an expert in the lore and story of the game Final Fantasy VII which includes the orignal game, remakes and spin-offs."},
        {"role": "user", "content": prompt}
    ]
)

# Print the response
print(completion.choices[0].message.content)

## Translation Functions

In [ ]:
def translate_dialogue(jp_text, official_en_text, context_info):
    prompt = f"""
    You are a localization assistant for Final Fantasy VII REBIRTH. Your task is to translate the following Japanese dialogue while keeping it faithful to the original meaning. Avoid excessive creative liberties, but ensure the translation sounds natural in English.

    Context:
    {context_info}

    Japanese Dialogue:
    {jp_text}

    Official English Localization:
    {official_en_text}

    Provide a revised, faithful English translation below:
    """

	# Call the Azure OpenAI API to generate a translation for the related dialogues
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {"role": "system", "content": "You are a localization expert."},
            {"role": "user", "content": prompt}
        ]
    )

    return completion.choices[0].message.content

## Load and Process Script Files

In [ ]:
# Load the JP and EN CSV files
df_jp = pd.read_csv("4000-MIDGR_TxtRes_jp.csv")
df_en = pd.read_csv("4000-MIDGR_TxtRes.csv")

# Merge files on ID to align translations
df_merged = df_jp.merge(df_en, on="id", suffixes=("_jp", "_en"))

# Group by related dialogue sets
df_merged["group"] = df_merged["id"].str.extract(r'(\$[^_]+_[^_]+_[^_]+)')

## Perform AI-Assisted Translation

In [ ]:
# Process each group
translated_data = []
for group_id, group in df_merged.groupby("group"):
    jp_text = "\n".join(group["text_jp"].tolist())
    official_en_text = "\n".join(group["text_en"].tolist())
    context_info = f"Scene ID: {group_id}"
    
    translated_text = translate_dialogue(jp_text, official_en_text, context_info)
    
    for idx, row in group.iterrows():
        translated_data.append({
            "id": row["id"],
            "original_jp": row["text_jp"],
            "official_en": row["text_en"],
            "revised_translation": translated_text.split('\n')[idx]
        })

# Convert translated data to DataFrame and save
df_translated = pd.DataFrame(translated_data)
df_translated.to_csv("4000-MIDGR_TxtRes_translated.csv", index=False)